In [26]:
import bw2data as bd
from collections import defaultdict
from tqdm import tqdm

In [3]:
bd.projects.set_current("ecoinvent-3.9-cutoff")

In [4]:
bd.databases

Databases dictionary with 4 object(s):
	Hospitals
	biosphere3
	ecoinvent-3.9-cutoff
	foreground-underspecification

In [5]:
bio = bd.Database("biosphere3")

In [6]:
ei = bd.Database("ecoinvent-3.9-cutoff")

In [59]:
n = [(x, x['reference product']) for x in ei if x['reference product'].startswith("nickel concentrate")]
n

[('nickel mine operation and benefication to nickel concentrate, 7% Ni' (kilogram, CN, None),
  'nickel concentrate, 7% Ni'),
 ('market for nickel concentrate, 7% Ni' (kilogram, CN, None),
  'nickel concentrate, 7% Ni'),
 ('cobalt production' (kilogram, GLO, None), 'nickel concentrate, 16% Ni'),
 ('nickel mine operation and benefication to nickel concentrate, 16% Ni' (kilogram, CA-QC, None),
  'nickel concentrate, 16% Ni'),
 ('market for nickel concentrate, 16% Ni' (kilogram, GLO, None),
  'nickel concentrate, 16% Ni'),
 ('market for nickel concentrate, 7% Ni' (kilogram, RoW, None),
  'nickel concentrate, 7% Ni'),
 ('nickel mine operation and benefication to nickel concentrate, 7% Ni' (kilogram, RoW, None),
  'nickel concentrate, 7% Ni')]

In [63]:
n[3][0]._data

{'comment': "This dataset represents the production of 1 kg of nickel ore, beneficiated to 16% Ni in Quebec (Canada) for the year 2010. \nThis dataset is a disagreggation of the existing dataset ''nickel mine operation, sulfidic ore'', which contains smelting and refining on top of the mine operation. The disagreggation was done  because primary data was available for nickel mine operation but not for subsequent stages of smelting and refining. Thus, the production volume of this dataset is already included in the global dataset ''nickel mine operation, sulfidic ore, GLO''. Until data on smelting and refining become available, this dataset cannot be used as part of a life cycle assessment.\nThe concentrate produced is composed of 16% Ni; 4% Cu; 0.24% Co; 0.61 g/t Au; 10.84 g/t Ag; 2.8 g/t Pt; 9.68 g/t Pd; 0.95 g/t Rh.\nThe sulfidic nickel ore is extracted in an underground mine, where 7.21 kg of ore are extracted per kg of concentrate produced. The recovery rate (metal in ore/metal in 

In [9]:
co2_codes = [x['code'] for x in bio if x['name'] == 'Carbon dioxide, fossil']

In [42]:
ch4_codes = [x['code'] for x in bio if x['name'] == 'Methane, fossil']

In [8]:
from bw2data.backends.proxies import ActivityDataset as AD, ExchangeDataset as ED

In [43]:
co2 = defaultdict(float)
ch4 = defaultdict(float)

In [49]:
qs = ED.select().where(ED.input_code << co2_codes, ED.output_database == "ecoinvent-3.9-cutoff")

In [50]:
qs2 = ED.select().where(ED.input_code << ch4_codes, ED.output_database == "ecoinvent-3.9-cutoff")

In [51]:
for exc in tqdm(qs):
    ds = bd.get_node(code=exc.output_code, database="ecoinvent-3.9-cutoff")
    scale = ds.rp_exchange()['production volume'] / ds.rp_exchange()['amount']
    co2[ds] += exc.data['amount'] * scale

100%|██████████████████████████████████████| 4950/4950 [00:05<00:00, 837.93it/s]


In [45]:
for exc in tqdm(qs2):
    ds = bd.get_node(code=exc.output_code, database="ecoinvent-3.9-cutoff")
    scale = ds.rp_exchange()['production volume'] / ds.rp_exchange()['amount']
    ch4[ds] += exc.data['amount'] * scale

100%|██████████████████████████████████████| 3254/3254 [00:03<00:00, 833.08it/s]


In [35]:
sum(co2.values()) / 1000 / 1e6

2133837.231490919

In [39]:
sum([x for x in co2.values() if x < 12438384881683])  / 1000 / 1e6

37748.65038912564

In [36]:
10132.0552271389 * 3.664

37123.850352236936

In [53]:
for x, y in sorted(zip(co2.values(), co2.keys()), reverse=True)[:20]:
    print("| {} | {} | {} |".format(y['name'], y['unit'], y['location']))

| heat production, natural gas, at boiler fan burner low-NOx non-modulating <100kW | megajoule | RoW |
| heat production, natural gas, at boiler atmospheric low-NOx non-modulating <100kW | megajoule | RoW |
| heat production, natural gas, at boiler atmospheric non-modulating <100kW | megajoule | RoW |
| heat production, natural gas, at boiler fan burner non-modulating <100kW | megajoule | RoW |
| heat production, natural gas, at boiler modulating <100kW | megajoule | RoW |
| heat production, natural gas, at boiler atm. low-NOx condensing non-modulating <100kW | megajoule | RoW |
| heat production, natural gas, at boiler condensing modulating <100kW | megajoule | RoW |
| heat production, at hard coal industrial furnace 1-10MW | megajoule | RoW |
| heat production, anthracite, at stove 5-15kW | megajoule | RoW |
| clinker production | kilogram | RoW |
| heat production, heavy fuel oil, at industrial furnace 1MW | megajoule | RoW |
| pig iron production | kilogram | RoW |
| heat productio

In [52]:
sorted(zip(co2.values(), co2.keys()), reverse=True)[:20]

[(315097449875768.1,
  'heat production, natural gas, at boiler fan burner low-NOx non-modulating <100kW' (megajoule, RoW, None)),
 (309418845420841.1,
  'heat production, natural gas, at boiler atmospheric low-NOx non-modulating <100kW' (megajoule, RoW, None)),
 (300901589765981.4,
  'heat production, natural gas, at boiler atmospheric non-modulating <100kW' (megajoule, RoW, None)),
 (300901589765981.4,
  'heat production, natural gas, at boiler fan burner non-modulating <100kW' (megajoule, RoW, None)),
 (295223082158125.06,
  'heat production, natural gas, at boiler modulating <100kW' (megajoule, RoW, None)),
 (283919999999999.94,
  'heat production, natural gas, at boiler atm. low-NOx condensing non-modulating <100kW' (megajoule, RoW, None)),
 (278187639233411.25,
  'heat production, natural gas, at boiler condensing modulating <100kW' (megajoule, RoW, None)),
 (12438384881683.525,
  'heat production, at hard coal industrial furnace 1-10MW' (megajoule, RoW, None)),
 (7657999263200.0

In [46]:
sum(ch4.values()) / 1e9

199.2571942515423

In [47]:
sorted(zip(ch4.values(), ch4.keys()), reverse=True)

[(44107119999.99984,
  'hard coal mine operation and hard coal preparation' (kilogram, CN, None)),
 (17230498342.2,
  'heat production, anthracite, at stove 5-15kW' (megajoule, RoW, None)),
 (11253480352.706005,
  'heat production, natural gas, at boiler fan burner low-NOx non-modulating <100kW' (megajoule, RoW, None)),
 (11050673050.744328,
  'heat production, natural gas, at boiler atmospheric low-NOx non-modulating <100kW' (megajoule, RoW, None)),
 (10800855000.0,
  'natural gas venting from petroleum/natural gas production' (cubic meter, GLO, None)),
 (10746485348.78505,
  'heat production, natural gas, at boiler atmospheric non-modulating <100kW' (megajoule, RoW, None)),
 (10746485348.78505,
  'heat production, natural gas, at boiler fan burner non-modulating <100kW' (megajoule, RoW, None)),
 (10543681505.647324,
  'heat production, natural gas, at boiler modulating <100kW' (megajoule, RoW, None)),
 (10139999999.999998,
  'heat production, natural gas, at boiler atm. low-NOx conde

In [29]:
exc.input_code, exc.input_database

('349b29d1-3e58-4c66-98b9-9d1a076efd2e', 'biosphere3')

In [13]:
co2_codes

['16eeda8a-1ea2-408e-ab37-2648495058dd',
 'e259263c-d1f1-449f-bb9b-73c6d0a32a00',
 'f9749677-9c9f-4678-ab55-c607dfdc2cb9',
 '349b29d1-3e58-4c66-98b9-9d1a076efd2e',
 'aa7cac3a-3625-41d4-bc54-33e2cf11ec46']